In [1]:
%pylab notebook
from time import sleep
from spi_helper import *
sys.path.append("../..")
from common import *

Populating the interactive namespace from numpy and matplotlib


# Connec to FPGA

Start remote litex server first
```bash
make server
```

In [2]:
r = conLitexServer('../build/csr.csv')

Could not connect to RemoteClient


# Play with GTX phy

In [128]:
# Resets sys and jesd clock domain
r.regs.ctrl_reset.write(1)
r.regs.ctrl_reset.read()

0

In [206]:
# Triggers the GTXInit state machine
r.regs.control_enable.write(1)
r.regs.control_phy_done.read()

1

In [204]:
# Transmit a square wave of frequency line_rate / 40
r.regs.phy0_transmitter_produce_square_wave.write(0)
r.regs.phy0_transmitter_produce_square_wave.read()

0

In [203]:
# Transmit pseudo random test pattern
r.regs.control_prbs_config.write(0)
r.regs.control_prbs_config.read()

0

In [208]:
r.regs.control_stpl_enable.write(0)
r.regs.control_stpl_enable.read()

0

In [201]:
# AD9174 --> to JESD204BLinkTX, triggers ILAS after CGS
r.regs.control_jsync.read()

0

In [202]:
r.regs.control_ready.read()

0

# Setup AD9174

In [2]:
r = None
ad = AdSpi(r)

reg PLL_STATUS (7b5) already exist (281)


In [16]:
ad.help(0x100)

reg 0x100 (DIG_RESET), bit 7:1 (RESERVED), R, reset 0x00
Reserved.

reg 0x100 (DIG_RESET), bit 0 (DIG_DATAPATH_PD), R/W, reset 0x01
Holds all digital logic (SERDES digital, digital clock generation, and digital datapath) in reset until clock tree is stable. 0: Normal operating mode. 1: Holds the digital logic in reset. Must be released (set to 0) after clocks to the chip are stable (PLL and DLL blocks are locked) to use the digital datapath.



In [184]:
# Power up sequence, Table 51
ad.wr(0x000, 0x81)  # Soft reset
ad.wr(0x000, 0x3C)  # 4 - wire SPI mode + ADDRINC
ad.wr(0x091, 0x00)  # Power up clock RX
ad.wr(0x206, 0x01)  # Enable PHYs
ad.wr(0x705, 0x01)  # Enable boot loader
ad.wr(0x090, 0x00)  # Power on DACs and bias supply

# Print product ID
print(hex(ad.rr(0x004)))
print(hex(ad.rr(0x005)))

0x74
0x91


In [185]:
# Disable DAC PLL and config for external clock, Table 52
ad.wr(0x095, 0x01)
ad.wr(0x790, 0xFF)
ad.wr(0x791, 0xFF)

ad.wr(0x008, (1 << 7) | (1 << 6)) # Select both DACs

# Magic numbers from Table 54 (calibration)
ad.wr(0x050, 0x2A)
ad.wr(0x061, 0x68)
ad.wr(0x051, 0x82)
ad.wr(0x051, 0x83)
print('CAL_STAT:', ad.rr(0x052))
ad.wr(0x081, 0x03)  # Power down calibration clocks

# JESD config, Table 55
ad.wr(0x100, 0x00)  # Power up digital datapath clocks
ad.wr(0x201, 0xF0)  # Power down the upper 4 PHYs
# ad.wr(0x201, 0xFF)  # Power down all PHYs.

ADC_CLK_DIVIDER_SHIFT = 6
# ADC clock output divider = /4
#   0: Divide by 1
#   1: Divide by 2
#   2: Divide by 3
#   3: Divide by 4
ad.wr(0x799, (3 << ADC_CLK_DIVIDER_SHIFT))

CAL_STAT: 1


0

In [186]:
# Delay Lock Loop (DLL) Configuration
ad.wr(0x0C0, 0x00)  # Power-up delay line.
ad.wr(0x0DB, 0x00)  # Update DLL settings to circuitry.
ad.wr(0x0DB, 0x01)
ad.wr(0x0DB, 0x00)
ad.wr(0x0C1, 0x68)  # set search mode for f_DAC > 4.5 GHz
ad.wr(0x0C1, 0x69)
ad.wr(0x0C7, 0x01)  # Enable DLL read status.
print('DLL locked:', ad.rr(0x0C3))

DLL locked: 1


In [187]:
# JESD204B Mode Setup
ad.wr(0x100, 0x00)
ad.wr(0x110, (0 << 5) | 8)  # JESD_MODE: Single link, mode 8
ad.wr(0x111, 0x21)   # DP_INTERP_MODE = 2x, CH_INTERP_MODE = 1x
print('Interpolation mode invalid:', (ad.rr(0x110) >> 7) & 0x01)
ad.wr(0x084, 1)  # SYSREF_PD: Power down reciever, we run subclass 0

ad.wr(0x300, 0b0001)  # select single link, page link0, enable link0
ad.wr(0x475, 0x09)  # Soft reset the JESD204B quad-byte deframer
ad.wr(0x453, (1 << 7) | 3)  # enable scrambler, L-value - 1
ad.wr(0x458, (0 << 5) | 15) # subclass 0, NP-value - 1
ad.wr(0x475, 1)  # Bring the JESD204B quad-byte deframer out of reset.

# Table 56: skipped as Channel interpolation mode = 1x

Interpolation mode invalid: 0


0

In [188]:
# Table 57: init MAin datapath
# TODO

In [189]:
# EQ settings for < 11 dB insertion loss
ad.wr(0x240, 0xAA)
ad.wr(0x241, 0xAA)
ad.wr(0x242, 0x55)
ad.wr(0x243, 0x55)
ad.wr(0x244, 0x1F)
ad.wr(0x245, 0x1F)
ad.wr(0x246, 0x1F)
ad.wr(0x247, 0x1F)
ad.wr(0x248, 0x1F)
ad.wr(0x249, 0x1F)
ad.wr(0x24A, 0x1F)
ad.wr(0x24B, 0x1F)
ad.wr(0x201, 0xF0)  # Power down the unused upper 4 PHYs
ad.wr(0x203, 0x01)  # Power down sync1
ad.wr(0x253, 0x01)  # Sync0 = LVDS
ad.wr(0x254, 0x01)  # Sync1 = LVDS

0

In [190]:
# SERDES required register write.
ad.wr(0x210, 0x16)
ad.wr(0x216, 0x05)
ad.wr(0x212, 0xFF)
ad.wr(0x212, 0x00)
ad.wr(0x210, 0x87)
ad.wr(0x216, 0x11)
ad.wr(0x213, 0x01)
ad.wr(0x213, 0x00)
ad.wr(0x200, 0x00)  # Power up the SERDES circuitry blocks.
sleep(0.1)

# SERDES required register write.
ad.wr(0x210, 0x86)
ad.wr(0x216, 0x40)
ad.wr(0x213, 0x01)
ad.wr(0x213, 0x00)
ad.wr(0x210, 0x86)
ad.wr(0x216, 0x00)
ad.wr(0x213, 0x01)
ad.wr(0x213, 0x00)
ad.wr(0x210, 0x87)
ad.wr(0x216, 0x01)
ad.wr(0x213, 0x01)
ad.wr(0x213, 0x00)
ad.wr(0x280, 0x05)
# Start up SERDES PLL circuitry blocks and initiate SERDES PLL calibration.
ad.wr(0x280, 0x01)
print('SERDES PLL locked:', ad.rr(0x281) & 0x01)

SERDES PLL locked: 1


In [182]:
ad.wr(0x03B, 0xF1)  # Enable the sync logic, and set the rotation mode to reset the synchronization logic upon a sync reset trigger.
ad.wr(0x03A, 0x02)  # Set up sync for one-shot sync mode.
ad.wr(0x300, 0b0001)  # select single link, page link0, enable link0

0

# Setup HMC7044
  * External VCO input
  * clkout12 = f_vco / 10 = 128 MHz

In [127]:
hmc = HmcSpi(r)

hmc.wr(0x000, 1)  # reset
hmc.wr(0x000, 0)

hmc.wr(0x054, 0)  # Disable SDATA driver (uni-direct. buffer)
hmc.wr(0x001, (1 << 6) | (1 << 5)) # High performance dividers / PLL

# VCO Selection
# 0 Internal disabled/external
# 1 High
# 2 Low
VCO_SELECT = 3
hmc.wr(0x003, (0 << VCO_SELECT))

# Enable output channel 12 and 13
hmc.wr(0x004, (1 << (12 // 2)))

# clkin1 as external VCO input
hmc.wr(0x005, (1 << 5))

# Magic numbers from datasheet Table 74
hmc.wr(0x09F, 0x4D)
hmc.wr(0x0A0, 0xDF)
hmc.wr(0x0A5, 0x06)
hmc.wr(0x0A8, 0x06)
hmc.wr(0x0B0, 0x04)

hmc.setupChannel(12, 10)  # DEV_CLK = 128 MHz
hmc.setupChannel(13, 80)  # SYSREF2 = 16 MHz

In [48]:
hmc.wr(0x000, 1)  # reset
hmc.wr(0x000, 0)

In [ ]:
r.close()